# Imports

In [1]:
from numpy.random import seed
seed(888)
import tensorflow
tensorflow.random.set_seed(404)

In [2]:
import os
import numpy as np
import tensorflow as tf

from time import strftime

from PIL import Image

# Constants

In [3]:
X_TRAIN_PATH= '/home/hp/Desktop/Udemy_Course/Tensorflow_Neural_Nets/original/digit_xtrain.csv'
X_TEST_PATH= '/home/hp/Desktop/Udemy_Course/Tensorflow_Neural_Nets/original/digit_xtest.csv'
Y_TRAIN_PATH= '/home/hp/Desktop/Udemy_Course/Tensorflow_Neural_Nets/original/digit_ytrain.csv'
Y_TEST_PATH= '/home/hp/Desktop/Udemy_Course/Tensorflow_Neural_Nets/original/digit_ytest.csv'

LOGGING_PATH = 'tensorboard_mnist_digit_logs/'


NR_CLASSES = 10

VALIDATION_SIZE=10000
IMAGE_WIDTH=28
IMAGE_HEIGHT = 28
CHANNELS = 1
TOTAL_INPUT = IMAGE_HEIGHT*IMAGE_WIDTH*CHANNELS

# Get the data

In [4]:
%%time

y_train_all = np.loadtxt(Y_TRAIN_PATH,delimiter=',',dtype=int)

CPU times: user 403 ms, sys: 7.86 ms, total: 411 ms
Wall time: 410 ms


In [5]:
y_train_all.shape

(60000,)

In [6]:
y_test = np.loadtxt(Y_TEST_PATH,delimiter=',',dtype=int)

In [7]:
%%time

x_train_all=np.loadtxt(X_TRAIN_PATH,delimiter=',',dtype =int)

CPU times: user 1min 5s, sys: 944 ms, total: 1min 6s
Wall time: 1min 6s


In [8]:
%%time

x_test = np.loadtxt(X_TEST_PATH,delimiter = ',',dtype=int)

CPU times: user 11 s, sys: 91.8 ms, total: 11.1 s
Wall time: 11.2 s


# Explore

In [9]:
x_train_all.shape

(60000, 784)

In [10]:
x_train_all[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   3,  18,  18,  18,
       126, 136, 175,  26, 166, 255, 247, 127,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,  30,  36,  94, 154, 17

In [11]:
y_train_all.shape

(60000,)

In [12]:
x_test.shape

(10000, 784)

# Data PreProcessing

In [13]:
# Rescale the features

x_train_all,x_test=x_train_all/255.0,x_test/255.0

#### Convert target values to one-hot encoding

In [14]:
y_train_all=np.eye(10)[y_train_all]

In [15]:
type(NR_CLASSES)

int

In [16]:
y_test = np.eye(NR_CLASSES)[y_test]


In [17]:
y_test.shape

(10000, 10)

## Create validation data from training data

In [18]:
x_val = x_train_all[:VALIDATION_SIZE]
y_val = y_train_all[:VALIDATION_SIZE]

In [19]:
x_train = x_train_all[VALIDATION_SIZE:]
y_train = y_train_all[VALIDATION_SIZE:]

In [20]:
y_train.shape

(50000, 10)

In [21]:
x_val.shape

(10000, 784)

# Setup Tensorflow Graph

In [22]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

Instructions for updating:
non-resource variables are not supported in the long term


In [23]:
X = tf.placeholder(tf.float32,shape=[None,TOTAL_INPUT])

In [24]:
Y = tf.placeholder(tf.float32,shape=[None,NR_CLASSES])

### Neural Network Architecture

#### Hyperparameters

In [25]:
nr_epochs = 50
learning_rate = 0.001

n_hidden1 = 512
n_hidden2 = 64

#### Hidden Layer 1 Setup:

In [26]:
def setup_layer(input,weight_dim,bias_dim,name):
    with tf.name_scope(name):
        ## Initializing weights and making weight variable
        initial_w = tf.truncated_normal(shape=weight_dim,stddev=0.1,seed = 42)
        w = tf.Variable(initial_value=initial_w)

        ## Initializing biases
        initial_b = tf.constant(value = 0.0,shape = [bias_dim])
        b = tf.Variable(initial_value = initial_b)

        layer_in = tf.matmul(input,w) + b
        
        if name == 'out':
            layer_out = tf.nn.softmax(layer_in)
        
        else:
            layer_out = tf.nn.relu(layer_in)
            
        tf.summary.histogram('weights',w)
        tf.summary.histogram('biases',b)
                
        return layer_out

In [27]:
#Model without dropout

# layer_1 = setup_layer(X,weight_dim=[TOTAL_INPUT,n_hidden1],bias_dim=n_hidden1,name='layer_1')

# layer_2 = setup_layer(layer_1,weight_dim=[n_hidden1,n_hidden2],bias_dim=n_hidden2,name='layer_2')

# output  = setup_layer(layer_2,weight_dim=[n_hidden2,NR_CLASSES],bias_dim=NR_CLASSES,name='out')

# model_name = f'{n_hidden1}-{n_hidden2} LR{learning_rate} Epoch{nr_epochs}'

In [28]:
layer_1 = setup_layer(X,weight_dim=[TOTAL_INPUT,n_hidden1],bias_dim=n_hidden1,name='layer_1')

layer_drop = tf.nn.dropout(layer_1,keep_prob=0.8,name='droput_layer')

layer_2 = setup_layer(layer_drop,weight_dim=[n_hidden1,n_hidden2],bias_dim=n_hidden2,name='layer_2')

output  = setup_layer(layer_2,weight_dim=[n_hidden2,NR_CLASSES],bias_dim=NR_CLASSES,name='out')

model_name = f'{n_hidden1}-DropOut-{n_hidden2} LR{learning_rate} Epoch{nr_epochs}'

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


# Tensorboard Setup

In [29]:
# Folder for tensorboard

folder_name = f'{model_name} at {strftime("%H:%M")}'
directory = os.path.join(LOGGING_PATH,folder_name)

try:
    os.makedirs(directory)
except OSError as exception:
    print(exception.strerror)
else:
    print("Successfully created directories")

Successfully created directories


# Loss, Optimization & Metrics

#### Defining loss function

In [30]:
with tf.name_scope('loss_calc'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits = output))

#### Defining Optimizer

In [31]:
with tf.name_scope('optimizer'):

    optimizer = tf.train.AdamOptimizer(learning_rate)
    train_step = optimizer.minimize(loss)

#### Accuracy Metric

In [32]:
with tf.name_scope('accuracy_calc'):

    correct_pred = tf.equal(tf.argmax(output,axis=1),tf.argmax(Y,axis=1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred , tf.float32))

In [33]:
## Summary is used to map the values to tensorboard
with tf.name_scope('performace'):

    tf.summary.scalar('accuracy:',accuracy)
    tf.summary.scalar('loss:',loss)

INFO:tensorflow:Summary name accuracy: is illegal; using accuracy_ instead.
INFO:tensorflow:Summary name loss: is illegal; using loss_ instead.


#### Check Input Images in Tensorboard

In [34]:
with tf.name_scope('show_image'):
    
    x_image = tf.reshape(X,[-1,28,28,1])
    tf.summary.image('image_input',x_image,max_outputs=4)

# Run Session

In [35]:
sess = tf.Session()

#### Setup FIlewriter and Merge Summaries

In [36]:
merged_summary = tf.summary.merge_all()

train_writer = tf.summary.FileWriter(directory+'/train')
train_writer.add_graph(sess.graph)

validation_writer = tf.summary.FileWriter(directory+'/validation')

#### initialise all the variables


In [37]:
init =tf.global_variables_initializer()
sess.run(init)

## Batching the data

In [38]:
size_of_batch = 1000

In [39]:
num_examples = y_train.shape[0]
nr_iterations = int(num_examples/size_of_batch) #make sure it is an integer

index_in_epoch = 0

In [40]:
def next_batch(batch_size, data, labels):
    
    global num_examples
    global index_in_epoch
    
    start = index_in_epoch
    index_in_epoch += batch_size
    
    if index_in_epoch > num_examples:
        start = 0
        index_in_epoch=batch_size
    
    end = index_in_epoch
    
    return data[start:end],labels[start:end]

# Training Loop

In [41]:
for epoch in range(nr_epochs):
    
                   ############# training ##################
    for i in range(nr_iterations):
        
        batch_x,batch_y= next_batch(batch_size=size_of_batch, data = x_train, labels = y_train) 
        
        feed_dictionary = {X:batch_x,Y:batch_y}

        sess.run(train_step,feed_dict = feed_dictionary)
        
    s , batch_accuracy = sess.run(fetches=[merged_summary,accuracy],feed_dict=feed_dictionary)
    
    train_writer.add_summary(s,epoch)
    
    print(f'Epoch {epoch} \t| training accuracy = {batch_accuracy}')
    
                 ############### Validation ####################
    
    summary = sess.run(fetches=merged_summary,feed_dict={X:x_val,Y:y_val})
    validation_writer.add_summary(summary,epoch)
    
print('Done Training')

Epoch 0 	| training accuracy = 0.8510000109672546
Epoch 1 	| training accuracy = 0.8610000014305115
Epoch 2 	| training accuracy = 0.8650000095367432
Epoch 3 	| training accuracy = 0.8690000176429749
Epoch 4 	| training accuracy = 0.8759999871253967
Epoch 5 	| training accuracy = 0.8730000257492065
Epoch 6 	| training accuracy = 0.9589999914169312
Epoch 7 	| training accuracy = 0.9800000190734863
Epoch 8 	| training accuracy = 0.9810000061988831
Epoch 9 	| training accuracy = 0.9750000238418579
Epoch 10 	| training accuracy = 0.9860000014305115
Epoch 11 	| training accuracy = 0.9810000061988831
Epoch 12 	| training accuracy = 0.9869999885559082
Epoch 13 	| training accuracy = 0.984000027179718
Epoch 14 	| training accuracy = 0.9900000095367432
Epoch 15 	| training accuracy = 0.9900000095367432
Epoch 16 	| training accuracy = 0.9879999756813049
Epoch 17 	| training accuracy = 0.9869999885559082
Epoch 18 	| training accuracy = 0.9890000224113464
Epoch 19 	| training accuracy = 0.98900002

# Make a Prediction

In [42]:
img = Image.open('/home/hp/Desktop/Udemy_Course/Tensorflow_Neural_Nets/MNIST/test_img.png')
img

In [43]:
#Converting it into a black and white image
bw = img.convert('L')

In [44]:
img_array = np.invert(bw)

In [45]:
img_array.shape

(28, 28)

In [46]:
#Flatten

test_img = img_array.ravel()
test_img.shape

(784,)

In [47]:
##Passing the flattened image to tensorflow to make predictions
prediction = sess.run(fetches=tf.argmax(output,axis=1),feed_dict = {X:[test_img]})

In [48]:
print(f'Prediction for test image is {prediction}')

Prediction for test image is [2]


# Testing and Evaluation

In [49]:
test_accuracy = sess.run(fetches=accuracy,feed_dict={X:x_test,Y:y_test})
print(f'Accuracy on test set: {test_accuracy:0.2%}')

Accuracy on test set: 97.91%


# Reset for the Next Run

In [50]:
train_writer.close()
validation_writer.close()
sess.close()
tf.reset_default_graph

<function tensorflow.python.framework.ops.reset_default_graph()>

# Cell for the first part of code

In [51]:
# with tf.name_scope('hidden_1'):

#     ## Initializing weights and making weight variable
#     initial_w1 = tf.truncated_normal([TOTAL_INPUT,n_hidden1],stddev=0.1,seed = 42)
#     w1 = tf.Variable(initial_value=initial_w1)

#     ## Initializing biases
#     initial_b1 = tf.constant(value = 0.0,shape = [n_hidden1])
#     b1 = tf.Variable(initial_value = initial_b1)

#     layer1_in = tf.matmul(X,w1) + b1 
#     layer1_out = tf.nn.relu(layer1_in)

#### Hidden Layer 2 Setup:

In [52]:
# with tf.name_scope('hidden_2'):

#     ## Initializing weights and making weight variable
#     initial_w2 = tf.truncated_normal([n_hidden1,n_hidden2],stddev=0.1,seed = 42)
#     w2 = tf.Variable(initial_value=initial_w2)

#     ## Initializing biases
#     initial_b2 = tf.constant(value = 0.0,shape = [n_hidden2])
#     b2 = tf.Variable(initial_value = initial_b2)

#     layer2_in = tf.matmul(layer1_out,w2) + b2
#     layer2_out = tf.nn.relu(layer2_in)

#### Output Layer Setup

In [53]:
# with tf.name_scope('output_layer'):

#     ## Initializing weights and making weight variable
#     initial_w3 = tf.truncated_normal([n_hidden2,NR_CLASSES],stddev=0.1,seed = 42)
#     w3 = tf.Variable(initial_value=initial_w3)

#     ## Initializing biases
#     initial_b3 = tf.constant(value = 0.0,shape = [NR_CLASSES])
#     b3 = tf.Variable(initial_value = initial_b3)

#     layer3_in = tf.matmul(layer2_out,w3) + b3
#     output = tf.nn.softmax(layer3_in)


In [54]:
#b3.eval(sess)